## Modeling Count Data with GLMs


A walk through fitting a Generalized Linear Model to data with a count response, and demonstrate that a custom implemented solution generates  identical estimates to those by the primary statistical modeling library in Python statsmodels [statsmodels](https://www.statsmodels.org/dev/index.html).       


In practice, we do not know the values of the proposed model's parameters, but we do know the data. We use the likelihood function to observe how the function changes for different parameter values while holding the data fixed. 
We can make use of this to judge which values of the parameters lead to greater relative chances for the sample to occur. Larger values of the likelihood correspond to values of the parameters that are relatively better supported by the data[2].
In short, *the underlying goal of a likelihood is to determine which parameters make the given data most likely*. 
   
The joint density of $n$ independently distributed observations $\mathbf{y} = (y_{1}, \cdots, y_{n})^{T}$ is given by:

$$
f(\mathbf{y}|\mathbf{\beta}) = \prod_{i=1}^{n} f_{i}(y_{i}|\mathbf{\beta})
$$

When this expression is interpreted as a function of unknown $\beta$ given known data $y$, we obtain the likelihood function:

$$
L(\mathbf{\beta}|\mathbf{y}) = \prod_{i=1}^{n} f_{i}(y_{i}|\mathbf{\beta})
$$

Solving the likelihood equation can be difficult. This can be partially alleviated by logging the likelihood expression, arriving at an expression for the log-likelihood:

$$
\mathcal{L}(\mathbf{\beta}|\mathbf{y}) = \sum_{i=1}^{n} f_{i}(y_{i}|\mathbf{\beta})
$$
             
<br>   

### Poisson Estimating Equations


A linear model can be fit by solving closed form equations. In the case of the Gaussian, we can maximize the log-likelihood and find an analytical solution directly. Unfortunately, that cannot be done with most Generalized Linear Models including Poisson regression. Instead, an iterative approach such as Iterative Reweighted Least Squares is used. The model is fit based on one-term Taylor linearization of the log-likelihood function identified as the working response. Given an initial estimate of the parameters $\hat{\beta}$, we calculate the estimated linear predictor $Ln(\hat{\mu}_{i}) = \hat{\eta}_{i} = x_{i}^{T}\beta$, which is then used to obtain the fitted values $\hat{\mu}_{i} = g^{-1}(\hat{\eta}_{i}) = exp(\hat{\eta}_{i})$. This process continues until the change in deviance between iterations falls below a predefined threshold. 
<br>   

Recall that the Poisson probability density function with mean $\mu$ is defined as:

$$
f(y) = \frac{\mu^{y} e^{-\mu}}{y!}
$$
<br>      

For a dataset with $n$ observations assumed to follow a Poisson distribution, with each observation having mean parameter $\mu_{i}$, the likelihood is given by

$$ 
L = \prod_{i=1}^{n} \frac{\mu_{i}^{y}e^{-\mu_{i}}}{y_{i}!},
$$

and similarly the log-likelihood as:

$$
\mathcal{L} = \sum_{i=1}^{n} y_{i} Ln(\mu_{i}) - \mu_{i} - Ln(y_{i}!)
$$



Every GLM has an associated [link function](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function), which specifies the relationship between the linear predictor and the mean of the distribution function. Each member of the exponential family (Gaussian, Binomial, Poisson, Gamma, Inverse Gaussian, Negative Binomial) has an associated canonical link, which, for the Poisson GLM is the *log link* (in GLM literature, the linear predictor is commonly represented by $\eta_{i}$ for models with canonical link):

$$
Ln(\mu_{i}) = \eta_{i} = x_{i}^{T}\beta
$$

<br>
  
Then the *inverse link* is the transformation required to obtain the mean. For Poisson model, this is the exponential function:   

$$
\mu_{i} = e^{\eta_{i}} = e^{x_{i}^{T}\beta}
$$

We can substitute this expression for $\mu_{i}$ into the Poisson log-likelihood equation, resulting in:

$$
\mathcal{L} = \sum_{i=1}^{n} \big(y_{i} (x_{i}^{T}\beta)  - exp(x_{i}^{T}\beta) - Ln(y_{i}!)\big)
$$


The gradient vector, or *score* is the first derivative of the log-likelihood function with respect to $\beta$:

$$
\frac{\partial \mathcal{L}}{\partial \beta} = \sum_{i=1}^{n} \big((y_{i} - exp(x_{i}^{T}\beta))x_{i}\big)
$$
<br>

The Hessian matrix is calculated as the second derivative of the log-likelihood function and is negative definite for $\beta$. It can be expressed as:

$$
\frac{\partial^{2} \mathcal{L}}{\partial \beta \partial \beta^{T}} = -\sum_{i=1}^{n} \big(exp(x_{i}^{T}\beta)\big)x_{i}x_{j}^{T}
$$


Estimation of the maximum likelihood covariance matrix is based on the negative inverse of the Hessian, $\Sigma$: 

$$
\sum = -H^{-1} = \Bigg[\sum_{i=1}^{n} \big(exp(x_{i}^{T}\beta)\big)x_{i}x_{j}^{T} \Bigg]^{-1}
$$
<br>

This is obtained from the last iteration of the estimation procedure.         
The square roots of the diagonal elements of $\Sigma$ are the values of parameter standard errors. 






# Symbol Map

- $n$: Number of dataset records    
<br>
- $p$: Number of model parameters excluding intercept ($p + 1$ *including* intercept)     
<br>    
- $X$: Model matrix  
<br>
- $y$: Dataset response     
<br>       
- $L = \prod_{i=1}^{n} \frac{\mu_{i}^{y}e^{-\mu_{i}}}{y_{i}!},$: Poisson likelihood                     
<br>                 
- $\mathcal{L} = \sum_{i=1}^{n} \big(y_{i} (x_{i}^{T}\beta)  - exp(x_{i}^{T}\beta) - Ln(y_{i}!)\big)$: Poisson log-likelihood         
<br>           
- $\beta = (\beta_{0}, \cdots, \beta_{p})^{T}$: Estimated model parameters     
<br>          
- $g(\mu_{i})$: Link function. For Poisson GLM, $g(\mu_{i}) = Ln(\mu_{i})$.              
<br>         
- $g'(\mu_{i}) = 1/\mu_{i}$          
<br>          
- $\eta = \mathbf{X\beta}$: Model linear predictor. Same as $g(\mu_{i})$.    
<br>             
- $\mu_{i} = g^{-1}(\eta_{i}) = g^{-1}(x_{i}^{T}\beta)$: Inverse link or mean function. For Poisson GLM, $\mu_{i} = exp(\eta_{i}) = exp(x_{i}^{T}\beta)$)     
<br>        
- $\frac{\partial \mathcal{L}}{\partial \beta} = U$: Gradient vector or score        
<br>      
- $\frac{\partial^{2} \mathcal{L}}{\partial \beta \partial \beta^{T}}$: Hessian matrix       
<br>     
- $\mathcal{I} =  -E\Big[\frac{\partial^{2} \mathcal{L}}{\partial \beta \partial \beta^{T}} \Big] = X^{T}WX$: Expected information matrix. For GLMs with canonical link, observed and expected information are the same.     
<br>         
- $w_{ii} = \frac{1}{var(Y_{i})}\big(\frac{\partial \mu_{i}}{\partial \eta_{i}}\big)^{2}$        
<br>        
- $W$: $n$-by-$n$ diagonal matrix of weights with $i^{th}$ element equal to $\mu_{i}$         
<br>           
- $\sum = -H^{-1} = \mathcal{I}^{-1}$: Variance-covariance matrix             
<br>   
- $\beta_{r+1} = \beta_{r} - H^{-1}U$: Newton-Raphson parameter update step     
<br>
- $\beta_{r+1} = [X^{T}WX]^{-1}X^{T}Wz$, where $z = X\beta_{r} + W^{-1}(y - \mu)$: IRLS update expression      
<br>       
- $z_{i} = \hat{\eta}_{i} + (y_{i} - \hat{\mu}_{i})g'(\mu)$: Working response, a one-term linearization of the log-likelihood function    
<br>          
- $\sum_{i=1}^{n}\big(y_{i} - exp(x_{i}^{T}\beta)\big)x_{i}^{T} = 0$: Poisson estimating equation         
<br>      
- The likelihood of the saturated model is 1.
<br>
- The log-likelihhod of the saturated model is 0. 
<br>



Fisher scoring has the advantage that it produces the asymptotic covariance matrix as a by-product.

The log mean is the natural parameter for the Poisson distribution, and the log link is the canonical link for a Poisson GLM. 
A 1-unit increase in $x_{j}$ has a multiplicative impact of $e^{\beta_{j}}$. The mean at $x_{j} + 1$ equals the mean at $x_{j}$ multiplied by $e^{\beta_{j}}$.   
<br>
Linear regression models are found by minimizing the sum of squared residuals. Poisson regression models are found by minimizing the sum of squared residual deviances, which is equivalent to maximizing the loglikelihood of the data given the model. 


### Model Assessment

**Deviance**: Measures how well the model fits the data. Is -2 * loglikelihood of the dataset given the model.       
**Deviance Residuals**: Array of -2 * loglikelihood for each data point.  

Null Deviance = 2(LL(Saturated Model) - LL(Null Model)) on df = df_Sat - df_Null

Residual Deviance = 2(LL(Saturated Model) - LL(Proposed Model)) df = df_Sat - df_Proposed

The Saturated Model is a model that assumes each data point has its own parameters (which means there are $n$ parameters to estimate.)

The Null Model assumes the exact "opposite", in that is assumes one parameter for all of the data points, which means the model only estimates one parameter.

The Proposed Model assumes you can explain your data points with $p$ parameters + an intercept term, so you have $p+1$ parameters.

If your Null Deviance is really small, it means that the Null Model explains the data pretty well. Likewise with your Residual Deviance. The null deviance shows how well the response is predicted by the model with nothing but an intercept.

What does really small mean? If your model is "good" then your Deviance is approx Chi^2 with (df_sat - df_model) degrees of freedom.

If you want to compare you Null model with your Proposed model, then you can look at

(Null Deviance - Residual Deviance) approx Chi^2 with df Proposed - df Null = (n-(p+1))-(n-1)=p


- degrees of freedom = no. of observations – no. of predictors    
<br>     
- Deviance of saturated model is 0, loglikelihood of saturated model will be the maximal attained value.       




<br>

The difference between the null deviance and the model's deviance is distributed as a chi-squared with degrees of freedom equal to the null df minus the model's df. For your model, that would be:

1-pchisq(256600 - 237230, df=(671266 - 671263))


By default, pchisq() gives the proportion of the distribution to the left of the value. To get the proportion more extreme than your difference, you can specify lower.tail = FALSE or subtract the result from 1 (as you and I have done).


Q-What hypothesis exactly are you testing with the statement 1-pchisq(256600 - 237230, df=(671266 - 671263))?


you are checking if the deviance changed more than might be expected by chance. Ie, you are testing if the model as a whole is better than the null model. It is analogous to the global F test in a linear model.


Q-The null hypothesis is 'the model as a whole is better than the null model', and you have rejected the null hypothesis, which means the model is poor?    

no the null hypothesis is: **the model as a whole is no better than the null model**. Since this has been rejected, we conclude that the data are not consistent with the null model. NB, this does not necessarily mean that our model is 'good' or 'correct'

Deviance is the likelihood ratio statistic comparing the saturated model to the model under consideration:

$$
D = 2\big(\mathcal{L}(\beta_{max}) - \mathcal{L}(\beta)\big)
$$

$D$ is approx chi-square with $m-p$ degrees of freedom, with expected value $m-p$. A value much greater than $m-p$ indicates a poor fitting model. 


Log-likelihood ratio statistic:

$$
\begin{align}
LRT&=2\big(\mathcal{L}(\beta_{unconstrained}) - \mathcal{L}(\beta_{constrained})\big) \\
   &=D_{constrained} - D_{unconstrained} \sim\chi^{2}_{q}
\end{align}
$$

Where $q$ represents the number of parameters in the unconstrained model not present in the constrained model. 


LRT Null hypothesis: $H_{0}$: Parameters in unconstrained model equal 0. 




We can compare the loglikelihoods of the models and select the model with the highest. But it would not be right to compare unadjusted loglikelihoods, because loglikelihood cannot decrease, and usually increases with the addition of variables, no matter how irrelevant they may be. We must charge a penalty for adding variables. 

**AIC Akaike Information Criterion**

$$
AIC = -2\mathcal{L} + 2q
$$

Where $q$ is the number of model parameters. 

The lower the AIC, the better the model. 


**Bayesian Information Criterion**

$$
BIC = -2\mathcal{L} + qLn(n)
$$

Where $n$ represents the number of observations, not the number of groups. 

In summary, for penalized loglikelihood tests, the improvement in loglikelihood for a model with $q$ additional parameters must be at least $2q$ for AIC or $qLn(n)$ for BIC. 







## IRLS Algorithm


$X$ is the n-by-p design matrix                        
$y$ is a n-by-1 vector representing the response            
$\text{offset}$ is a n-by-1 vector representing the exposure              
$\mu$ is a n-by-1 vector representing the mean of each observation      
$\eta$ is a n-by-1 vector representing the linear predictor/component, $\eta = Ln(\mu)$        
$W$ is a n-by-n diagonal matrix with each value equal to $\mu$ (weight matrix)     
$z$ is a n-by-1 vector representing the working response               
$\epsilon$ represents the change in deviance below which iteration will terminate      
<br>     


####  Pseudocode


- $\text{deviance} = 0$      
- $\mu = \text{mean(y)}$    
- $\eta = Ln(\mu)$    
- $epsilon = .0001$   
- $\Delta \text{deviance}$ = $\mathrm{Inf}$
<br>    



**WHILE** $|\Delta \text{deviance}| \gt \epsilon$:

$\qquad W$ = diag($\mu$)
<br>    
$\qquad z$ = $\eta + \frac{y - \mu}{\mu} - \text{offset}$
<br>    
$\qquad \beta = (X^{T}WX)^{-1}X^{T}Wz$   
<br>
$\qquad \eta = X\beta + \text{offset}$      
<br>
$\qquad \mu = exp(\eta)$                      
<br>
$ \qquad \text{deviance}_{0} = \text{deviance}$
<br>  
$ \qquad \text{deviance} = 2 \sum \big(yLn(y/\mu) - (y-\mu)\big)$
<br>  
$ \qquad \Delta \text{deviance} = |\text{deviance} - \text{deviance}_{0}|$
<br>


In [ ]:
"""
IRLS pre-processing and setup. 
"""
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from numpy.random import RandomState
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import special
from scipy import stats
from numpy.linalg import inv

pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 50000)
np.set_printoptions(suppress=True, precision=5)
pd.options.mode.chained_assignment = None

dforiginal      = pd.read_csv("U:/Repos/LTC/Incidence_Model/Datasets/dfgrpd.csv")
cat_fields      = ["GENDER", "AGE_CATEGORY"]
dfinit_         = dforiginal[cat_fields + ["INCIDENCE_MNTH", "EXPOSURE"]]
dfinit          = pd.get_dummies(dfinit_, columns=cat_fields, drop_first=True)
dfinit.columns  = [i.upper() for i in dfinit.columns]
response_fields = ["INCIDENCE_MNTH", "EXPOSURE"]
design_fields   = list(set(dfinit.columns).difference(set(response_fields)))
dfgrpd          = dfinit.groupby(design_fields, as_index=False).sum()
offset          = dfgrpd.EXPOSURE.values.reshape(dfgrpd.EXPOSURE.values.size, 1)
Xinit           = dfgrpd[design_fields]
Xintercept      = np.ones(Xinit.shape[0]).reshape(Xinit.shape[0], 1)
X               = np.hstack([Xintercept, Xinit])
y               = dfgrpd.INCIDENCE_MNTH.values.reshape(Xinit.shape[0], 1)


def deviance(y, mu):
    """
    Compute Poisson deviance.
    """
    y  = y.ravel().tolist()
    mu = mu.ravel().tolist()
    def f(yi, mui):
        return(yi * np.log(yi / mui) - (yi - mui))
    v = np.asarray([0 if (i==0 or j==0) else f(i, j)  for i, j in zip(y, mu)])
    return(2 * v.sum())


def likelihood(y, mu):
    """
    Compute Poisson likelihood.
    """
    v = y * np.log(mu) - mu - np.log(special.factorial(y))
    return(np.exp(v).sum())

                                     
def loglikelihood(y, mu):
    """
    Compute Poisson log-likelihood.
    """
    y  = y.ravel()
    mu = mu.ravel()
    def f(yi, mui):
        return(yi * np.log(mui) - mui - np.log(special.factorial(yi)))
    v = np.asarray([0 if i==0 else f(i, j) for i, j in zip(y, mu)])
    return(v.sum())



coeffs = list()  
loglik = list()
devlst = list()
tol    = .00001
ddev   = np.Inf
mu0    = (y + y.mean()) / 2
eta0   = np.log(mu0)
mu0    = np.exp(eta0)  
dev0   = deviance(y=y, mu=mu0)
llk0   = loglikelihood(y=y, mu=mu0)
fsi    = 0           # Fisher scoring iteration counter


In [ ]:
"""
Original implementation of Iterative Reweighted Least Squares (IRLS).
Author: James D. Triveri
Date  : 2019-02
"""
# Append initial deviance, loglikelihood and parameter estimates.
devlst.append(dev0); loglik.append(llk0)



while np.abs(ddev) > tol:
    
    fsi+=1
    
    # Compute updated W, working response, coeffs, linear component and mean.
    W   = np.diag(np.ones(X.shape[0])) * mu0
    z   = (eta0 + ((y - mu0) / mu0) - np.log(offset)).reshape(y.size, 1)
    B   = inv((X.T @ W @ X)) @ (X.T @ W @ z)
    eta = X @ B.reshape(B.size, 1) + np.log(offset)
    mu  = np.exp(eta).reshape(eta.size, 1)
    
    # Compute updated deviance.
    dev  = deviance(y=y, mu=mu)
    llk  = loglikelihood(y=y, mu=mu)
    ddev = dev0 - dev
    vcov = inv((X.T @ W @ X))
    mu0, eta0, llk0, dev0 = mu, eta, llk, dev

    # Append updated parameters, loglikelihood and deviance.
    devlst.append(dev0); loglik.append(llk0); coeffs.append(B)

    
print("\n[=== Summary =====================================================]\n")
print(f"Fisher Scoring Iterations: {fsi}\n")
print("Deviance:\n")
for j in enumerate(devlst): print(j)
print("")
print("Loglikelihood:\n")
for k in enumerate(loglik): print(k)
print("")
print("Variance-Covariance Matrix:\n")
print(vcov)
print("\n[=================================================================]\n")
print("")


### Summary Output Description

- **`VARIABLE`**: The name of the estimated parameter.          
<br>
- **`ESTIMATE`**: The parameter estimate/model coefficient resulting from the running of IRLS.              
<br>
- **`STDERROR`**: The uncertainty in the parameter estimate.     
<br>
- **`Z-VALUE`**: Represents the ratio of `ESTIMATE / STDERROR`.     
<br>
- **`P(>|z|)`**: Probability of observing the outcome if the coefficient was not significantly different from 0.   
<br>


In [ ]:
##### Model Coefficients and Standard Errors ######
params = coeffs[-1].ravel()

dfparams = pd.DataFrame({
    "VARIABLE":["INTERCEPT"] + Xinit.columns.tolist(),
    "ESTIMATE":coeffs[-1].T.ravel(),
    "STDERROR":np.sqrt(np.diagonal(vcov)),
    "Z-VALUE" :(coeffs[-1].T.ravel() / np.sqrt(np.diagonal(vcov))),
    "P(>|z|)" :stats.norm.cdf((coeffs[-1].T.ravel() / np.sqrt(np.diagonal(vcov))))
    })

dfparams




In [ ]:
##### Other Summary Statistics #####
# Deviance is approx ChiSquare with m - p df
# Computation of null deviance (intercept only model).
mu_null = np.ones(y.size) * y.ravel().mean()

resid_deviance = devlst[-1]
null_deviance = deviance(y=y.ravel(), mu=mu_null)
deviance

In [ ]:

##### Plot log-likelihood as a function of Fisher Scoring iteration #####
x0, y0 = zip(*enumerate(loglik))

plt.figure(figsize=(10, 7))
plt.scatter(x0, y0, marker="o", edgecolor="#000000", s=50, color="#FFFFFF")
plt.title("Log-likelihood as a Function of Scoring Iteration", color="#000000", loc="left")
plt.plot(x0, y0, color="#FF0000", linewidth=1.5)
plt.xlabel("Fisher Scoring Iteration")
plt.ylabel("Log-likelihood")
plt.grid(True)
plt.show()

In [ ]:
##### Plot Deviance as a function of Fisher Scoring Iteration #####
x1, y1 = zip(*enumerate(devlst))

plt.figure(figsize=(10, 7))
plt.scatter(x1, y1, marker="o", edgecolor="#000000", s=50, color="#FFFFFF")
plt.title("Poisson Deviance as a Function of Scoring Iteration", color="#000000", loc="left")
plt.plot(x1, y1, color="#FF0000", linewidth=1.5)
plt.xlabel("Fisher Scoring Iteration")
plt.ylabel("Deviance")
plt.grid(True)
plt.show()


In [ ]:
##### Statsmodels Poisson GLM fit #####

keep_fields  = ["GENDER", "AGE_CATEGORY", "INCIDENCE_MNTH", "EXPOSURE"]
dfmodel      = dforiginal[keep_fields]
dfgrpd2      = dfmodel.groupby(["GENDER", "AGE_CATEGORY"], as_index=False).sum()
formula_expr = "INCIDENCE_MNTH ~ C(GENDER) + C(AGE_CATEGORY)"

poisson_mdl  = smf.glm(
    formula=formula_expr, data=dfgrpd2, 
    family=sm.families.Poisson(link=sm.families.links.log),
    exposure=dfgrpd2["EXPOSURE"]
    ).fit()


# Bind reference to estimated model coefficients.
params = poisson_mdl.params
summ   = poisson_mdl.summary()



In [ ]:
poisson_mdl.deviance




In [ ]:
#poisson_mdl.null_deviance
dir(poisson_mdl)

In [ ]:
yy     = dfgrpd2.INCIDENCE_MNTH.values / dfgrpd2.EXPOSURE.values
munull = yy.mean()


loglikelihood(y=yy, mu=munull)
